In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
emb = nn.Embedding(10000, 20, padding_idx=0)
inp = torch.tensor([1, 2, 5, 2, 10], dtype=torch.int64)
out = emb(inp)
print(out)
print(out.size())

tensor([[ 1.7727, -0.3389, -0.1840,  0.5645, -0.0775, -0.2983, -1.2093,  0.4237,
          2.0949,  0.8848, -0.7230, -0.2230,  0.0488, -0.2833, -1.4652,  1.5824,
         -1.2820,  0.5358, -1.1885, -0.1016],
        [-0.6956,  1.3488,  0.7089, -0.1382,  0.6998,  0.2334,  0.5400, -0.1324,
          0.8616,  0.0209, -0.2070,  0.2540, -1.8005,  0.1579, -0.8390, -0.0417,
         -0.2288, -0.0323, -0.5640,  1.7304],
        [ 0.0446, -1.3939,  0.9387, -0.8969,  1.5750,  1.2532,  0.7739, -1.4090,
          1.5641,  0.7083, -0.9670,  1.0753, -0.9089,  0.6813,  3.0262, -0.2384,
          0.4439,  0.1285, -0.6647, -1.1171],
        [-0.6956,  1.3488,  0.7089, -0.1382,  0.6998,  0.2334,  0.5400, -0.1324,
          0.8616,  0.0209, -0.2070,  0.2540, -1.8005,  0.1579, -0.8390, -0.0417,
         -0.2288, -0.0323, -0.5640,  1.7304],
        [-0.0880, -0.4981, -0.4940,  1.6404, -0.0312, -0.0764, -0.5956,  1.2367,
         -0.9138,  0.7083,  0.2669, -1.4221,  2.2204, -0.0848, -1.3911,  0.4804,
      

In [3]:
import glob
import pathlib
import re

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]<.*?>")
shift_marks_regex = re.compile("([?!])")

def text2ids(text, vocab_dict):
    text = remove_marks_regex.sub("", text)
    text = shift_marks_regex.sub(r" \1 ", text)
    tokens = text.split()
    return [vocab_dict.get(token, 0) for token in tokens]

def list2tensor(token_idxes, max_len=100, padding=True):
    if len(token_idxes) > max_len:
        token_idxes = token_idxes[:max_len]
    n_tokens = len(token_idxes)
    if padding:
        token_idxes = token_idxes + [0] * (max_len - len(token_idxes))
    return torch.tensor(token_idxes, dtype=torch.int64), n_tokens

In [4]:
class IMDBDataset(Dataset):
    def __init__(self, dir_path, train=True, max_len=100, padding=True):
        self.max_len = max_len
        self.padding = padding
        path = pathlib.Path(dir_path)
        vocab_path = path.joinpath("imdb.vocab")
        
        self.vocab_array = vocab_path.open().read().strip().splitlines()
        self.vocab_dict = dict((w, i+1) for (i, w) in enumerate(self.vocab_array))
        
        if train:
            target_path = path.joinpath("train")
        else:
            target_path = path.joinpath("test")
        pos_files = sorted(glob.glob(str(target_path.joinpath("pos/*.txt"))))
        neg_files = sorted(glob.glob(str(target_path.joinpath("neg/*.txt"))))
        self.labeled_files = \
            list(zip([0]*len(neg_files), neg_files)) + \
            list(zip([1]*len(pos_files), pos_files))
        
    @property
    def vocab_size(self):
        return len(self.vocab_array)
    
    def __len__(self):
        return len(self.labeled_files)
    
    def __getitem__(self, idx):
        label, f = self.labeled_files[idx]
        data = open(f).read().lower()
        data = text2ids(data, self.vocab_dict)
        data, n_tokens = list2tensor(data, self.max_len, self.padding)
        return data, label, n_tokens

In [5]:
train_data = IMDBDataset("./data/aclImdb/")
test_data = IMDBDataset("./data/aclImdb/", train=False)
train_loader = DataLoader(train_data, batch_size=32,
                          shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=32,
                         shuffle=False, num_workers=8)

In [6]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size,
                            num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        if l is not None:
            x = x[list(range(len(x))), l-1, :]
        else:
            x = x[:, -1, :]
        x = self.linear(x)
        x = x.squeeze()
        return x

In [7]:
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [8]:
from statistics import mean

net = SequenceTaggingNet(train_data.vocab_size+1, num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoc in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        l = l.to("cuda:0")
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print("epoc: {}\tmean(losses): {:.3f}\ttrain_acc: {:.3f}\tval_acc: {:.3f}".format(
    epoc, mean(losses), train_acc, val_acc))

  0%|          | 0/782 [00:00<?, ?it/s]

epoc: 0	mean(losses): 0.676	train_acc: 0.629	val_acc: 0.607


  0%|          | 0/782 [00:00<?, ?it/s]

epoc: 1	mean(losses): 0.603	train_acc: 0.742	val_acc: 0.698


  0%|          | 0/782 [00:00<?, ?it/s]

...

epoc: 7	mean(losses): 0.234	train_acc: 0.937	val_acc: 0.780


  0%|          | 0/782 [00:00<?, ?it/s]

epoc: 8	mean(losses): 0.193	train_acc: 0.946	val_acc: 0.773


100%|██████████| 782/782 [00:09<00:00, 83.17it/s]


epoc: 9	mean(losses): 0.162	train_acc: 0.963	val_acc: 0.770


In [9]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

train_X, train_y = load_svmlight_file("./data/aclImdb/train/labeledBow.feat")
test_X, test_y = load_svmlight_file("./data/aclImdb/test/labeledBow.feat")

model = LogisticRegression(C=0.1, max_iter=5000)
model.fit(train_X[:, :test_X.shape[1]], train_y[:test_y.shape[0]])
train_score = model.score(train_X[:, :test_X.shape[1]], train_y[:test_y.shape[0]])
test_score = model.score(test_X, test_y)
print(train_score, test_score)

0.93132 0.394


In [10]:
class SequenceTaggingNet2(SequenceTaggingNet):
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        x, h = self.lstm(x, h0)
        if l is not None:
            hidden_state, cell_state = h
            x = hidden_state[-1]
        else:
            x = x[:, -1, :]
        
        x = self.linear(x).squeeze()
        return x

In [11]:
def eval_net2(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        l, sort_idx = torch.sort(l, descending=True)
        x = x[sort_idx]
        y = y[sort_idx]
        
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [12]:
net = SequenceTaggingNet2(train_data.vocab_size+1, num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        l, sort_idx = torch.sort(l, descending=True)
        x = x[sort_idx]
        y = y[sort_idx]
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        l = l.to("cuda:0")
        
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net2(net, train_loader, "cuda:0")
    val_acc = eval_net2(net, test_loader, "cuda:0")
    print("epoch: {}\tmean(losses): {:.3f}\ttrain_acc: {:.3f}\tval_acc: {:.3f}".format(
    epoch, mean(losses), train_acc, val_acc))

  0%|          | 0/782 [00:00<?, ?it/s]

epoch: 0	mean(losses): 0.688	train_acc: 0.542	val_acc: 0.543


  0%|          | 0/782 [00:00<?, ?it/s]

epoch: 1	mean(losses): 0.681	train_acc: 0.592	val_acc: 0.561


  0%|          | 0/782 [00:00<?, ?it/s]

...

epoch: 7	mean(losses): 0.296	train_acc: 0.916	val_acc: 0.771


  0%|          | 0/782 [00:00<?, ?it/s]

epoch: 8	mean(losses): 0.249	train_acc: 0.921	val_acc: 0.760


100%|██████████| 782/782 [00:10<00:00, 72.33it/s]


epoch: 9	mean(losses): 0.205	train_acc: 0.952	val_acc: 0.767


### 文章生成

In [13]:
import string
all_chars = string.printable
print(all_chars)
vocab_size = len(all_chars)
vocab_dict = dict((c, i) for i, c in enumerate(all_chars))

def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]

def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [14]:
class ShakespeareDataset(Dataset):
    def __init__(self, path, chunk_size=200):
        data = str2ints(open(path).read().strip(), vocab_dict)
        data = torch.tensor(data, dtype=torch.int64).split(chunk_size)
        if len(data[-1]) < chunk_size:
            data = data[:-1]
        self.data = data
        self.n_chunks = len(self.data)
        
    def __len__(self):
        return self.n_chunks
    
    def __getitem__(self, idx):
        return self.data[idx]

In [15]:
ds = ShakespeareDataset("./data/tinyshakespeare.txt")
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=8)

In [16]:
class SequenceGenerationNet(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        x = self.linear(x)
        return x, h

In [17]:
def generate_seq(net, start_phrase="The King said ", length=200,
                 temperature=0.8, device="cpu"):
    net.eval()
    result = []
    
    start_tensor = torch.tensor(str2ints(start_phrase, vocab_dict),
                                dtype=torch.int64).to(device)
    x0 = start_tensor.unsqueeze(0)
    o, h = net(x0)
    out_dist = o[:, -1].view(-1).exp()
    top_i = torch.multinomial(out_dist, 1)[0]
    result.append(top_i)
    
    for i in range(length):
        inp = torch.tensor([[top_i]], dtype=torch.int64)
        inp = inp.to(device)
        o, h = net(inp, h)
        out_dist = o.view(-1).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        result.append(top_i)
        
    return start_phrase + ints2str(result, all_chars)

In [18]:
import numpy
s = numpy.array([[1, 2], [3, 4]])
print(s[:, -1].reshape(-1))

[2 4]


In [19]:
from statistics import mean

net = SequenceGenerationNet(vocab_size, 20, 50, num_layers=2, dropout=0.1)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
    net.train()
    losses = []
    for data in tqdm.tqdm(loader):
        x = data[:, : -1]
        y = data[:, 1:]
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        
        y_pred, _ = net(x)
        loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
        
    print("epoch:{}\tmean(losses):{:.3f}".format(epoch, mean(losses)))
    with torch.no_grad():
        print(generate_seq(net, device="cuda:0"))

  0%|          | 0/175 [00:00<?, ?it/s]

epoch:0	mean(losses):3.490
The King said :de' mwTliain
nITp hciorfni
a`nhWbtOiRedtouc ,lrn Whbe:htao2metfCnn htsaboh:;rl  hm,'wh,s abAd n
 i ;Hbe dbdo  I ,lahnhetteeh+o o, adrmbs leh Peao:loekRos tpsoetHLAroR
a fLypedIfieked
a ioIdIiiOto ,aro


  0%|          | 0/175 [00:00<?, ?it/s]

epoch:1	mean(losses):3.225
The King said t, taer admn ch t
recmngreit! erordraeim g hko sbod Lmouewh,f nto tsno mEfg oy nsy hi.oeoaw ed
Mi i
Ae iaLb li, yb wCshde ahwi sccyshhscyalop esawkemo rotwh
. lo  hsl yu t d?ydturMplayr
e IIt  maesfon 


  0%|          | 0/175 [00:00<?, ?it/s]

...

epoch:47	mean(losses):1.686
The King said and in vieple, and the'chelds;
Where dean his pariss talk the gave but to cheace there
what up with enteoune parwar Lord lick Pay
Even so ip: I heeg in the juce to make my lords
Nou all thich that wela


  0%|          | 0/175 [00:00<?, ?it/s]

epoch:48	mean(losses):1.683
The King said curnen som lay featot them.
Make seal not a kibt is grace, and I love
Tuves and holy say with haw parn my lord,
My tome miseling beaster frummy.

LADY BAFDE:
That father alver me, her sir, it their hai


100%|██████████| 175/175 [00:02<00:00, 59.48it/s]


epoch:49	mean(losses):1.679
The King said was banith!

CLOMENORIO:
And as not this wold I holse weel: should drows:
And sains bebit of the fruster: strive
But you it chance, and not blegnicuponts.

HENRY BOLWARCE:
What take vincence!

KING RIC


### Encoder-Decoderモデルによる機械翻訳

In [20]:
import re
import collections
import itertools

remove_marks_regex = re.compile("[\,\(\)\[\]\*:;¿¡]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
    text = remove_marks_regex.sub("", text)
    text = shift_marks_regex.sub(r" \1 ", text)
    return text
    
def parse_line(line):
    line = normalize(line.strip())
    src, trg, othor = line.split("\t")
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens

def build_vocab(tokens):
    counts = collections.Counter(tokens)
    sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
    word_list = ["<UNK>", "<SOS>", "<EOS>"] + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
    words = words + ["<EOS>"]
    words = [word_dict.get(w, 0) for w in words]
    seq_len = len(words)
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    return torch.tensor(words,dtype=torch.int64), seq_len

In [21]:
class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        def filter_pair(p):
            return not (len(p[0]) > max_len or len(p[1]) > max_len)
        
        with open(path) as fp:
            pairs = map(parse_line, fp)
            pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        '''
        pairs = []
        with open(path) as fp:
            lines = fp.readlines()
            for line in lines:
                pair = map(parse_line, line)
                pair = filter(filter_pair, pair)
                pairs.append(list(pair))
        '''
        src = [p[0] for p in pairs]
        trg = [p[1] for p in pairs]
        self.src_word_list, self.src_word_dict = \
            build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = \
            build_vocab(itertools.chain.from_iterable(trg))
        self.src_data = [words2tensor(words, self.src_word_dict, max_len)
                         for words in src]
        self.trg_data = [words2tensor(words, self.trg_word_dict, max_len, -100)
                         for words in trg]
        
    def __len__(self):
        return len(self.src_data)
    
    def __getitem__(self, idx):
        src, lsrc = self.src_data[idx]
        trg, ltrg = self.trg_data[idx]
        return src, lsrc, trg, ltrg

In [22]:
batch_size = 64
max_len = 10
path = "./data/spa-eng/spa.txt"
ds = TranslationPairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=8)

In [23]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        _, h = self.lstm(x, h0)
        return h

In [24]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x = nn.utils.rnn.pad_packed_sequence(
                x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h

In [25]:
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, ds.src_word_dict,
                                         max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    seq_len = [seq_len]
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    
    ctx = enc(input_tensor, l=seq_len)
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        o, h = dec(z.view(1, 1), h)
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        z = wi
    return " ".join(ds.trg_word_list[i] for i in results)

In [26]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("I am a student.", enc, dec)

'pulpo pulpo caliéntese ubicaron ubicaron ubicaron ubicaron rescatado rescatado caminamos defendí carcajadas malgastado alegres alegres'

In [27]:
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)
loss_f = nn.CrossEntropyLoss()

In [28]:
def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)

for epoch in range(30):
    enc.train(), dec.train()
    losses = []
    for x, lx, y, ly in tqdm.tqdm(loader):
        lx, sort_idx = lx.sort(descending=True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x, y = x.to("cuda:0"), y.to("cuda:0")
        ctx = enc(x, l=lx)
        
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        h0 = (ctx[0][:, sort_idx, :], ctx[1][:, sort_idx, :])
        z = y[:, :-1].detach()
        z[z==-100] = 0
        
        o, _ = dec(z, h0, l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:, 1:max(ly)]).squeeze())
        enc.zero_grad(), dec.zero_grad()
        loss.backward()
        opt_enc.step(), opt_dec.step()
        losses.append(loss.item())
        
    enc.eval(), dec.eval()
    print(epoch, mean(losses))
    with torch.no_grad():
        print(translate("I am a student.",
                        enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("He likes to eat pizza.",
                        enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("She is my mother.",
                        enc, dec, max_len=max_len, device="cuda:0"))

  0%|          | 0/1666 [00:00<?, ?it/s]

0 5.3914137713763175
un gran .
que tom se se se se se se se se
mi padre .


  0%|          | 0/1666 [00:00<?, ?it/s]

1 3.38359257181724
un gran .
que tom se se se se se quedó .
mi padre .


  0%|          | 0/1666 [00:00<?, ?it/s]

...

27 0.44919021558217787
.
comer como el tonto .
mi madre .


  0%|          | 0/1666 [00:00<?, ?it/s]

28 0.4406930407484611
.
comer como un tonto .
mi madre .


100%|██████████| 1666/1666 [00:31<00:00, 53.70it/s]

29 0.4322027559755515
.
comer como un tonto .
mi madre .
